<a href="https://colab.research.google.com/github/ccosmin97/idc-prostate-mri-analysis/blob/main/DEMO_PROSTATE158_nnunet_prostate_mri_us_biopsy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prostate segmentation on IDC collection -- Prostate
*   Dataset : [Prostate-MRI-US-BIOPSY]()
*   Goal : Prostate segmentation using Task24 Prostate nnU-net pre-trained model, T2 input single-modality

# Download platimatch binaries

In [1]:
!wget -O plastimatch.zip https://www.dropbox.com/scl/fi/c2nsgewjw1ppp5nc75oiv/plastimatch.zip?rlkey=ya25vx7bojz7xobqbsr74ujxy&dl=0

--2024-06-13 03:44:40--  https://www.dropbox.com/scl/fi/c2nsgewjw1ppp5nc75oiv/plastimatch.zip?rlkey=ya25vx7bojz7xobqbsr74ujxy
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4f467f60d3c2e91da4153aa49c.dl.dropboxusercontent.com/cd/0/inline/CUtECldVksgUcUnQzCj3stS5f818ZcGm_9UuteEfvdnguMBkmwdnF6djYlo7xrGOd9rahQ7SfF1lIVQKzD9wKCS_7QNoXK8cc3JUyUzP4NqNtiuQexasD8Xc134NrukwpqtcyGfhpPdz9nqzDsZgu6-I/file# [following]
--2024-06-13 03:44:41--  https://uc4f467f60d3c2e91da4153aa49c.dl.dropboxusercontent.com/cd/0/inline/CUtECldVksgUcUnQzCj3stS5f818ZcGm_9UuteEfvdnguMBkmwdnF6djYlo7xrGOd9rahQ7SfF1lIVQKzD9wKCS_7QNoXK8cc3JUyUzP4NqNtiuQexasD8Xc134NrukwpqtcyGfhpPdz9nqzDsZgu6-I/file
Resolving uc4f467f60d3c2e91da4153aa49c.dl.dropboxusercontent.com (uc4f467f60d3c2e91da4153aa49c.dl.dropboxusercontent.com)... 162.125.65.15, 26

## Install Python packages

In [2]:
!git clone https://github.com/herrmannlab/highdicom.git
#!cd highdicom && python setup.py install
!cd highdicom && pip install .

Cloning into 'highdicom'...
remote: Enumerating objects: 7651, done.
remote: Counting objects: 100% (1240/1240), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 7651 (delta 1006), reused 1139 (delta 964), pack-reused 6411
Receiving objects: 100% (7651/7651), 3.86 MiB | 6.71 MiB/s, done.
Resolving deltas: 100% (5061/5061), done.
Processing /content/highdicom
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 kB 13.3 MB/s eta 0:00:00
  Created wheel for highdicom: filename=highdicom-0.22.0-py3-none-any.whl size=825743 sha256=3327d6b8074f7844c350364d63d5dcf17c9e83f83626eaf86db336d11e33b485
  Stored in directory: /tmp/pip-ephem-wheel-cache-2kfg4vf7/wheels/85/80/88/55471f8e67c8ff2f09dd805b1c472392006bfa74851bec2b6d
Successfully built highdicom


In [3]:
!pip install pydicom
!pip install seg-metrics
!pip install nibabel
!pip install dcm2niix
!pip install SimpleITK
!pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.4/451.4 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dcm2niix: filename=dcm2niix-1.0.20220715-cp310-cp310-linux_x86_64.whl size=559552 sha256=b4b73f550ea27fa9ceae37e30237ce9d103e6b8e0755913a1657684e3c400506
  Stored in directory: /root/.cache/pip/wheels/88/8d/9b/5ea20c0451a1acddef585757be7dfec121ee076e58503b267c
Successfully built dcm2niix
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 41.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to b

In [4]:
!pip install -U "monai[all]" pyyaml munch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.3 MB/s eta 0:00:00
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.5/266.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━

In [7]:
# @title
!unzip plastimatch.zip

Archive:  plastimatch.zip
   creating: plastimatch_build/
  inflating: plastimatch_build/libplmregister.a  
  inflating: plastimatch_build/dicom_info  
  inflating: plastimatch_build/libplmsys.a  
  inflating: plastimatch_build/hnd_to_pfm  
  inflating: plastimatch_build/libinih.a  
  inflating: plastimatch_build/libspecfun.a  
  inflating: plastimatch_build/cmake_uninstall.cmake  
  inflating: plastimatch_build/CPackConfig.cmake  
  inflating: plastimatch_build/libplmreconstruct.a  
  inflating: plastimatch_build/cmake_install.cmake  
  inflating: plastimatch_build/plm_config.h  
  inflating: plastimatch_build/libplmclp.a  
  inflating: plastimatch_build/libnkidecompress.a  
  inflating: plastimatch_build/plastimatch  
  inflating: plastimatch_build/xvi_archive  
  inflating: plastimatch_build/dlib_train  
  inflating: plastimatch_build/Makefile  
  inflating: plastimatch_build/run_lcov.sh  
  inflating: plastimatch_build/CPackSourceConfig.cmake  
  inflating: plastimatch_build/liblbf

Check what system dependencies are missing:

In [8]:
!ldd plastimatch_build/plastimatch

	linux-vdso.so.1 (0x00007fff7f3e8000)
	libITKIOLSM-4.13.so.1 => not found
	libITKIOBruker-4.13.so.1 => not found
	libITKIOHDF5-4.13.so.1 => not found
	libm.so.6 => /lib/x86_64-linux-gnu/libm.so.6 (0x00007aaa9b1e5000)
	libITKIOMINC-4.13.so.1 => not found
	libITKIOMRC-4.13.so.1 => not found
	libdcmjpeg.so.16 => not found
	libdlib.so.19 => not found
	libfftw3.so.3 => not found
	libdcmimgle.so.16 => not found
	libdcmdata.so.16 => not found
	liboflog.so.16 => not found
	libofstd.so.16 => not found
	libITKIOBMP-4.13.so.1 => not found
	libITKIOGDCM-4.13.so.1 => not found
	libITKIOGIPL-4.13.so.1 => not found
	libITKIOJPEG-4.13.so.1 => not found
	libITKIOMeta-4.13.so.1 => not found
	libITKIONIFTI-4.13.so.1 => not found
	libITKIONRRD-4.13.so.1 => not found
	libITKIOPNG-4.13.so.1 => not found
	libITKIOTIFF-4.13.so.1 => not found
	libITKIOVTK-4.13.so.1 => not found
	libITKIOBioRad-4.13.so.1 => not found
	libITKIOStimulate-4.13.so.1 => not found
	libITKOptimizers-4.13.so.1 => not found
	libITKIOGE-

Install the packages that solve these dependencies:

In [9]:
!sudo apt update && sudo apt install libinsighttoolkit4-dev libdcmtk16 libdlib19 libfftw3-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,900 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,173 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe 

In [10]:
# @title
!pip install itk

In [11]:
!mv plastimatch_build/plastimatch ./plastimatch
!rm -r plastimatch_build
!mv

mv: missing file operand
Try 'mv --help' for more information.


In [12]:
!cp ./plastimatch /usr/local/bin/plastimatch

# Global variables

## For SR and radiomics computation

In [13]:
#nnunet code mapping
import pandas as pd
#radiomics df
!wget -N https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_shape_features_code_mapping.csv
nnunet_shape_features_code_mapping_df = pd.read_csv("nnunet_shape_features_code_mapping.csv")

--2024-06-13 03:48:48--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_shape_features_code_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1053 (1.0K) [text/plain]
Saving to: ‘nnunet_shape_features_code_mapping.csv’

nnunet_shape_featur 100%[===================>]   1.03K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-06-13 03:48:48 (88.7 MB/s) - ‘nnunet_shape_features_code_mapping.csv’ saved [1053/1053]



In [14]:
nnunet_segments_code_mapping_df = pd.DataFrame.from_dict({'segment': ['Prostate'],
                                                          'Finding_CodingSchemeDesignator': ['SCT'], 'Finding_CodeValue' : ['123037004'], 'Finding_CodeMeaning' : ['Anatomical Structure'],
                                                          'FindingSite_CodingSchemeDesignator' : ['SCT'], 'FindingSite_CodeValue' : ['41216001'], 'FindingSite_CodeMeaning' : ['Prostate']})
nnunet_segments_code_mapping_df.head()

,segment,Finding_CodingSchemeDesignator,Finding_CodeValue,Finding_CodeMeaning,FindingSite_CodingSchemeDesignator,FindingSite_CodeValue,FindingSite_CodeMeaning
0,Prostate,SCT,123037004,Anatomical Structure,SCT,41216001,Prostate


In [15]:
idc_seg_segments_code_mapping_df = pd.DataFrame.from_dict({'segment': ['Prostate', 'Peripheral zone of the prostate', 'Transition zone of the prostate'],
                                                          'Finding_CodingSchemeDesignator': ['SCT', 'SCT', 'SCT'], 'Finding_CodeValue' : ['123037004', '123037004', '123037004'],
                                                           'Finding_CodeMeaning' : ['Anatomical Structure', 'Anatomical Structure', 'Anatomical Structure'],
                                                          'FindingSite_CodingSchemeDesignator' : ['SCT', 'SCT', 'SCT'],
                                                           'FindingSite_CodeValue' : ['41216001', '279706003', '399384005'],
                                                           'FindingSite_CodeMeaning' : ['Prostate', 'Peripheral zone of the prostate', 'Transition zone of the prostate']})
idc_seg_segments_code_mapping_df.head()

,segment,Finding_CodingSchemeDesignator,Finding_CodeValue,Finding_CodeMeaning,FindingSite_CodingSchemeDesignator,FindingSite_CodeValue,FindingSite_CodeMeaning
0,Prostate,SCT,123037004,Anatomical Structure,SCT,41216001,Prostate
1,Peripheral zone of the prostate,SCT,123037004,Anatomical Structure,SCT,279706003,Peripheral zone of the prostate
2,Transition zone of the prostate,SCT,123037004,Anatomical Structure,SCT,399384005,Transition zone of the prostate


## Variables used for resampling -- inference -- setup labelID for ground truth segs

In [16]:
# IDC collection for paths setup
collection_name = "prostatex" #up to user
##nnunet
model_type = '3d_fullres'#other options are '2d',..
nnunet_labelID = 1 #unique values for nifti segmentation volumes predicted, here 1 is prostate and 0 is background
desired_grid = "T2" #Other option is "ADC"

## Global paths

Here we create the necessary folders, IDC folders for download MR T2 data, and nnunet workspace.

In [17]:
import os
#qin_prostate_repeatibility
os.environ["prostatex_root"] = os.path.join(os.getcwd(), "prostateX")
os.environ["prostatex_dicom"] = os.path.join(os.environ["prostatex_root"], "dicom")
os.environ["prostatex_nii"] = os.path.join(os.environ["prostatex_root"], "nii")
os.environ["prostatex_dicom_seg"] = os.path.join(os.environ["prostatex_root"], "dicom_seg")
os.environ["prostatex_nii_seg"] = os.path.join(os.environ["prostatex_root"], "nii_seg")
os.environ["prostatex_nii_seg_resampled"] = os.path.join(os.environ["prostatex_root"], "nii_seg_resampled")
os.environ["prostatex_nii_seg_resampled_formatted"] = os.path.join(os.environ["prostatex_root"], "prostatex_nii_seg_resampled_formatted")
os.environ["prostatex_nii_formatted"] = os.path.join(os.environ["prostatex_root"], "nii_formatted")
os.environ["prostatex_nrrd"] = os.path.join(os.environ["prostatex_root"], "nrrd")
#SEG IDC
#seg files
os.environ["stl_labelmap_root"] = os.path.join(os.getcwd(), "stl_labelmap")
os.environ["stl_labelmap_dicom_seg"] = os.path.join(os.environ["stl_labelmap_root"], "dicom_seg")
os.environ["stl_labelmap_nii_seg"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg")
os.environ["stl_labelmap_nii_seg_sr_json"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_sr_json")
os.environ["stl_labelmap_nii_seg_sr"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_sr")
os.environ["stl_labelmap_nii_seg_formatted"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_formatted")
os.environ["stl_labelmap_nii_seg_resampled"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_resampled")
os.environ["stl_labelmap_nii_seg_features"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_features")
os.environ["stl_labelmap_nii_seg_split_temp_resampled"] = os.path.join(os.environ["stl_labelmap_root"], "nii_seg_split_temp_resampled")
#monai
os.environ["monai_workspace"] = "monai_workspace"
os.environ["monai_workspace_eval"] = os.path.join(os.environ["monai_workspace"], "eval")
os.environ["monai_workspace_eval_resampled"] = os.path.join(os.environ["monai_workspace"], "eval_resampled/")
os.environ["monai_workspace_eval_resampled_split"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_split/")
os.environ["monai_workspace_eval_resampled_split_labelID"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_split_labelID")
os.environ["monai_workspace_eval_resampled_dcm"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_dcm/")
os.environ["monai_workspace_eval_resampled_dcm_to_nifti"] = os.path.join(os.environ["monai_workspace"], "eval_resampled_dcm_to_nifti/")
os.environ["monai_workspace_eval_features"] = os.path.join(os.environ["monai_workspace"], "eval_features")
os.environ["monai_workspace_eval_split_temp"] = os.path.join(os.environ["monai_workspace"], "temp_split")
os.environ["monai_workspace_eval_sr"] = os.path.join(os.environ["monai_workspace"], "structured_reports")
os.environ["monai_workspace_eval_sr_json"] = os.path.join(os.environ["monai_workspace"], "structured_reports_json")

os.environ["monai_workspace_dataset"] = os.path.join(os.environ["monai_workspace"], "dataset/")
os.environ["monai_workspace_model_dir"] = os.path.join(os.environ["monai_workspace"], "models")
os.environ["monai_workspace_model_path"] = os.path.join(os.environ["monai_workspace"], "models", "anatomy.pt")
#path where model pre-trained weights are stored
# os.environ["PATH_TO_MODEL_FILE"] = os.path.join(os.environ["nnUNet"], "models", "Task024_Prostate.zip")
# #misc
# os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_DL")
# os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_DL", "Sorted")
# os.environ["logs"] = os.path.join(os.getcwd(), "logs")
###UPDATED
#idc data
os.environ["IDC_Downloads"] = os.path.join(os.getcwd(), "IDC_Downloads")
os.environ["IDC_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_Downloads_Sorted")
os.environ["IDC_IMG_Downloads"] = os.path.join(os.getcwd(), "IDC_IMG_Downloads")
os.environ["IDC_IMG_Downloads_Sorted"] = os.path.join(os.getcwd(), "IDC_IMG_Downloads_Sorted")
#evaluation
os.environ["mri_us_biopsy_analysis"] = os.path.join(os.getcwd(), "mri_us_biopsy_analysis")
os.environ["mri_us_biopsy_analysis_results"] = os.path.join(os.environ["mri_us_biopsy_analysis"], "results")
os.environ["mri_us_biopsy_analysis_verbose"] = os.path.join(os.environ["mri_us_biopsy_analysis"], "results_verbose")
os.environ["logs"] = "logs"
#create dirs for specific folders names
for key, path in os.environ.items():
  check_patterns = [True for el in ["prostatex", "nnunet", "IDC", "nnUNet", "stl_labelmap", "monai_workspace_model_dir",
                                    "logs", "RESULTS_FOLDER", "monai_workspace_eval", "monai_workspace_dataset",
                                    "stl_labelmaps", "mri_us_biopsy_analysis"] if el in key]
  if True in check_patterns:
    !mkdir -p $path

# Setup of the Colab VM



In the following cells we will confirm you have a GPU before doing anything else, and will install and import all the Python dependencies.

The main python packages we need to install are:
* `nnunet` - which is the [codebase for the nn-UNet framework](https://github.com/MIC-DKFZ/nnUNet) we are going to be using for the segmentation step;
* `pydicom`, a Python [package](https://github.com/pydicom/pydicom) that lets the use read, modify, and write DICOM data in an easy "pythonic" way - that we are going to use to distinguish different DICOM objects from each other.

## GPU checks

In [18]:
# check wether the Colab Instance was correctly initialized with a GPU instance
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

if not has_gpu:
  print("Your Colab VM does not have a GPU - check \"Runtime > Change runtime type\"")

In [19]:
# check which model of GPU the notebook is equipped with - a Tesla K80 or T4
# T4 is the best performing on the two - and can about half the GPU processing time

!nvidia-smi

Thu Jun 13 03:48:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   35C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install command-line tools


[Plastimatch](https://plastimatch.org/index.html) is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/denbonte/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one).

In [20]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [21]:
!echo $(plastimatch --version)

plastimatch version 1.9.4-46-g950dde17


[dcmqi](https://github.com/QIICR/dcmqi) is an open source library that can help with the conversion between imaging research formats and the standard DICOM representation for image analysis results. More specifically, you can use dcmqi convert DICOM Segmentation objects (DICOM SEG) into research formats, such as NIfTI and NRRD.

In [22]:
%%capture
!wget https://github.com/QIICR/dcmqi/releases/download/v1.2.5/dcmqi-1.2.5-linux.tar.gz
!tar zxvf dcmqi-1.2.5-linux.tar.gz
!cp dcmqi-1.2.5-linux/bin/* /usr/local/bin/

Finally, we are going to install [Subversion](https://subversion.apache.org/), a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [23]:
%%capture

!sudo apt install subversion

In [24]:
!echo $(svn --version | head -n 2)

svn, version 1.14.1 (r1886195) compiled May 21 2022, 10:52:35 on x86_64-pc-linux-gnu


Download monai bundle from github

In [25]:
# !svn checkout https://github.com/Project-MONAI/model-zoo/trunk/models/prostate_mri_anatomy monai_prostate158/
!git clone https://github.com/Project-MONAI/model-zoo/

Cloning into 'model-zoo'...
remote: Enumerating objects: 4437, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 4437 (delta 20), reused 30 (delta 12), pack-reused 4380
Receiving objects: 100% (4437/4437), 36.04 MiB | 30.15 MiB/s, done.
Resolving deltas: 100% (3004/3004), done.


Next we set up few things to help with visualization of the segmentations later.

In [26]:
import os
import sys
import shutil
import csv
import random

import os
import glob
import csv
import json

import nibabel as nib

import time
import gdown

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk
from SimpleITK import GetArrayViewFromImage as ArrayView
from functools import partial

import seg_metrics.seg_metrics as sg
# from medpy.metric.binary import dc as dice_coef
# from medpy.metric.binary import hd as hausdorff_distance
# from medpy.metric.binary import asd as asd
# from medpy.metric.binary import precision
# from medpy.metric.binary import recall
# from medpy.metric.binary import specificity
# from medpy.metric.binary import sensitivity
# from medpy.metric.binary import true_positive_rate

import radiomics
from radiomics import featureextractor

# from medpy.filter.binary import largest_connected_component

# use the "tensorflow_version" magic to make sure TF 1.x is imported
# %tensorflow_version 2.x
# import tensorflow as tf
# import keras

from monai.bundle import ConfigParser, run

In [27]:
# Packages for the structured report

import highdicom

from pathlib import Path

import highdicom as hd

from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion,
    SourceImageForMeasurement,
    SourceImageForMeasurementGroup
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    MeasurementsAndQualitativeEvaluations,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation,
    ImageLibrary,
    ImageLibraryEntryDescriptors
)
from highdicom.sr.value_types import (
    CodedConcept,
    CodeContentItem,
)

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

In [28]:
# PyPlastimatch - python wrapper for Plastimatch (and interactive notebook visualisation)
!svn checkout https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch pyplastimatch

svn: E170013: Unable to connect to a repository at URL 'https://github.com/AIM-Harvard/pyplastimatch/trunk/pyplastimatch'
svn: E160013: '/AIM-Harvard/pyplastimatch/trunk/pyplastimatch' path not found


In [29]:
# dicomsort is the pythong package that can sort DICOM files into
# folder organization based on user-specified DICOM attributes
!git clone https://github.com/pieper/dicomsort.git

Cloning into 'dicomsort'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 169 (delta 23), reused 33 (delta 16), pack-reused 126
Receiving objects: 100% (169/169), 87.84 KiB | 725.00 KiB/s, done.
Resolving deltas: 100% (86/86), done.


In [30]:
!pip install pyplastimatch

In [31]:
from pyplastimatch.utils.install import install_precompiled_binaries
install_precompiled_binaries()

PyPlastimatch Plastimatch installation utility.
NOTE: this utility is not meant to be replace the normal install of Plastimatch via apt.
Rather, it is meant to be used in case a Plastimatch binary is not available for a specific distribution.

System distribution: Ubuntu 22.04

Matching distribution found in the latest PyPlastimatch release.


Installing binaries... Done.
Installing dependencies... Done.


In [32]:
from pyplastimatch import pyplastimatch as pypla

# Custom functions

## Pre-processing / post-processing

In [33]:
def download_idc_data_serie_uid_seg(df_in, out_path, idc_downloads_path, idc_downloads_sorted_path):
  # save the list of GCS URLs into a file
  selection_manifest = os.path.join(idc_downloads_path, "idc_manifest.txt")
  df_in["gcs_url"].to_csv(selection_manifest, header=False, index=False)
  # let's make sure the download folder is clean, in case you ran this cell earlier
  # for a different dataset
  # !rm -rf {os.environ["IDC_Downloads"]+"/*.dcm"}
  !cat {selection_manifest} | gsutil -m cp -I {idc_downloads_path}
  !python dicomsort/dicomsort.py -k -u {idc_downloads_path} {idc_downloads_sorted_path}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
  # !rm -rf {os.environ["qin_prostate_rep_dicom"]+"/*"}
  in_mv = idc_downloads_sorted_path+'/*'
  !mv $in_mv $out_path
  #{os.environ["qin_prostate_rep_dicom"]}
  # convert_dcm_sorted(input_path=os.environ["qin_prostate_rep_dicom"],
  #                 output_path=os.environ["qin_prostate_rep_root"], idc_df=selection_df)

In [34]:
def resample_idc_seg_nii(input_folder_seg_nii, output_folder_seg_nii, search_path_ref_nii_file):
  for seg_nii in glob.glob(os.path.join(input_folder_seg_nii, "*.nii.gz")):
    print(f"{desired_grid}_path found : {search_path_ref_nii_file}")
    print(f"pred path : {seg_nii}")
    output = os.path.join(output_folder_seg_nii, f"{seg_nii.split('/')[-1]}")
    resample_args_to_t2_origin = {"input" : seg_nii,#change to pred path if no largest_component_retrieval necessary
                          "output" : output,
                          "fixed" : search_path_ref_nii_file,
                          "interpolation" : "nn"}
    print(f"Resampling seg named : {seg_nii.split('/')[-1]} with fixed grid : {search_path_ref_nii_file.split('/')[-1]} ...")
    pypla.resample(verbose = False, **resample_args_to_t2_origin)

In [35]:
def resample_idc_seg_nii(input_folder_seg_nii, output_folder_seg_nii, search_path_ref_nii_file):
  for seg_nii in glob.glob(os.path.join(input_folder_seg_nii, "*.nii.gz")):
    print(f"{desired_grid}_path found : {search_path_ref_nii_file}")
    print(f"pred path : {seg_nii}")
    output = os.path.join(output_folder_seg_nii, f"{seg_nii.split('/')[-1]}")
    resample_args_to_t2_origin = {"input" : seg_nii,#change to pred path if no largest_component_retrieval necessary
                          "output" : output,
                          "fixed" : search_path_ref_nii_file,
                          "interpolation" : "nn"}
    print(f"Resampling seg named : {seg_nii.split('/')[-1]} with fixed grid : {search_path_ref_nii_file.split('/')[-1]} ...")
    pypla.resample(verbose = False, **resample_args_to_t2_origin)

In [36]:
def change_labelID_seg_nii(in_seg, labelID, output_folder_path):
  input_labelmap_image = sitk.ReadImage(in_seg)
  input_labelmap_arr = sitk.GetArrayFromImage(input_labelmap_image)
  temp_arr = input_labelmap_arr
  temp_arr[temp_arr != 0] = labelID
  # Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
  res_img = sitk.GetImageFromArray(temp_arr)
  res_img.CopyInformation(input_labelmap_image)
  sitk.WriteImage(res_img, os.path.join(output_folder_path,in_seg.split('/')[-1]))

In [37]:
def combine_prostate_regions(input_path_nii_pred, output_path_folder, filename):
  # Starting with SimpleITK
  img_pred = sitk.ReadImage(input_path_nii_pred)
  npy_pred = sitk.GetArrayFromImage(img_pred)
  # Custom Python code working on a numpy array.
  npy_pred[npy_pred != 0] = 1
  # Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
  res_img = sitk.GetImageFromArray(npy_pred)
  res_img.CopyInformation(img_pred)
  # Save output
  sitk.WriteImage(res_img, os.path.join(output_path_folder, filename))

In [38]:
def download_idc_data_serie_uid_seg(df_in, out_path, idc_downloads_path, idc_downloads_sorted_path):
  # save the list of GCS URLs into a file
  selection_manifest = os.path.join(idc_downloads_path, "idc_manifest.txt")
  df_in["gcs_url"].to_csv(selection_manifest, header=False, index=False)
  # let's make sure the download folder is clean, in case you ran this cell earlier
  # for a different dataset
  # !rm -rf {os.environ["IDC_Downloads"]+"/*.dcm"}
  !cat {selection_manifest} | gsutil -m cp -I {idc_downloads_path}
  !python dicomsort/dicomsort.py -k -u {idc_downloads_path} {idc_downloads_sorted_path}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
  # !rm -rf {os.environ["qin_prostate_rep_dicom"]+"/*"}
  in_mv = idc_downloads_sorted_path+'/*'
  !mv $in_mv $out_path
  #{os.environ["qin_prostate_rep_dicom"]}
  # convert_dcm_sorted(input_path=os.environ["qin_prostate_rep_dicom"],
  #                 output_path=os.environ["qin_prostate_rep_root"], idc_df=selection_df)

In [39]:
def convert_seg_to_nii(input_path, output_path):
  if not os.path.exists(output_path):
    !mkdir -p $output_path

  print(f'input path : {input_path}')
  print(f'output_path : {output_path}')
  !segimage2itkimage --inputDICOM $input_path --outputDirectory $output_path \
  --outputType nii

In [40]:
def split_seg_nii(input_labelmap, output_folder_path, dic_labelID):
  for labelMeaning, unique_label in dic_labelID.items():
    input_labelmap_image = sitk.ReadImage(input_labelmap)
    input_labelmap_arr = sitk.GetArrayFromImage(input_labelmap_image)
    temp_arr = input_labelmap_arr
    temp_arr[temp_arr != int(unique_label)] = 0
    temp_arr[temp_arr == int(unique_label)] = 1
    # Converting back to SimpleITK (assumes we didn't move the image in space as we copy the information from the original)
    res_img = sitk.GetImageFromArray(temp_arr)
    res_img.CopyInformation(input_labelmap_image)
    sitk.WriteImage(res_img, os.path.join(output_folder_path, f"{labelMeaning}.nii.gz"))

In [41]:
def resample_idc_seg_nii(input_folder_seg_nii, output_folder_seg_nii, search_path_ref_nii_file):
  for seg_nii in glob.glob(os.path.join(input_folder_seg_nii, "*.nii.gz")):
    print(f"{desired_grid}_path found : {search_path_ref_nii_file}")
    print(f"pred path : {seg_nii}")
    output = os.path.join(output_folder_seg_nii, f"{seg_nii.split('/')[-1]}")
    resample_args_to_t2_origin = {"input" : seg_nii,#change to pred path if no largest_component_retrieval necessary
                          "output" : output,
                          "fixed" : search_path_ref_nii_file,
                          "interpolation" : "nn"}
    print(f"Resampling seg named : {seg_nii.split('/')[-1]} with fixed grid : {search_path_ref_nii_file.split('/')[-1]} ...")
    pypla.resample(verbose = True, **resample_args_to_t2_origin)

In [42]:
def resample_itk_identity(mask_path : str,
                          image_path : str, out_image_path : str):
  image = sitk.ReadImage(image_path)
  mask = sitk.ReadImage(mask_path)
  result_image = sitk.Resample(mask, image, sitk.Transform(),sitk.sitkNearestNeighbor, 0, mask.GetPixelID())
  sitk.WriteImage(result_image, out_image_path)

In [43]:
def convert_image_dcm_to_nii(input_path, output_path_root, prefix="", format="nii"):
  """
  Conversion of DICOM MR data to NIFTI using dcm2niix
  input_path : str, folder containing DICOM instances .dcm
  output_path_root : str, output folder
  prefix : str, prefix for output file name
  """
  if not os.path.exists(output_path_root):
    !mkdir -p $output_path_root
  !dcm2niix -z y -m y -f %i_{prefix} -o $output_path_root $input_path

In [44]:
def convert_image_dcm_to_nrrd(input_path, output_path_root, format="nrrd", prefix=""):
  if not os.path.exists(output_path_root):
    !mkdir -p $output_path_root
  # !dcm2niix -e y -z y -m y -f %i_{output_name_file_suffix} -o $output_path_root $input_path
  out_path_file = f"{output_path_root}/{prefix}.nrrd"
  !plastimatch convert \
  --input $input_path \
  --output-img $out_path_file

In [45]:
def convert_dcm_sorted(input_path,output_path, idc_df):
  """
  Convert DCM MR sorted by PatID/StudyUID/SerieUID in NIFTI format
  input_path : str, root folder containing all series sorted by patid/studyUID/serieUID/sopUID.dcm
  output_path : str, root folder of where to store converted MR T2 series
  idc_df : pandas.Dataframe, contains IDC metadata to retrieve modality information etc per serieUID
  """
  for serie_folder in sorted(glob.glob(os.path.join(input_path, "**", "**", "*"))):#grab serie folder in sorted MR IDC DICOM data folder
    path_serie_dcm_lst = glob.glob(os.path.join(serie_folder, "*.dcm"))
    modality = idc_df[idc_df["SeriesInstanceUID"] == path_serie_dcm_lst[0].split('/')[-2]]["Modality"].iloc[0]#get modality per serie
    seriesInstanceUID = serie_folder.split("/")[-1]
    studyInstanceUID = serie_folder.split("/")[-2]
    patientID = serie_folder.split("/")[-3]
    print(f"Serie processed : {serie_folder}")
    print(f"SeriesDescription : {pydicom.read_file(glob.glob(os.path.join(serie_folder, '*.dcm'))[0]).SeriesDescription}")
    print(f"Modality : {pydicom.read_file(glob.glob(os.path.join(serie_folder, '*.dcm'))[0]).Modality}")
    #convert to NIFTI
    convert_image_dcm_to_nii(input_path=serie_folder,
                           output_path_root=os.path.join(output_path, "nii"),
                           prefix=f"{seriesInstanceUID}", format="nii")
    convert_image_dcm_to_nrrd(input_path=serie_folder,
                           output_path_root=os.path.join(output_path, "nrrd"),
                           prefix=f"{patientID}_{seriesInstanceUID}", format="nrrd")

In [46]:
def download_idc_data_serie_uid(idc_t2_df, idc_downloads_path, idc_downloads_sorted_path, idc_dicom_path, idc_nii_path):
  """
  Download IDC data from gcs_urls retrieved from BigQuery
  idc_df : pandas.Dataframe, contains MR series DICOM metadata and gcs_urls
  """
  # save the list of GCS URLs into a file
  selection_manifest = os.path.join(idc_downloads_path, "idc_manifest.txt")
  idc_t2_df["gcs_url"].to_csv(selection_manifest, header=False, index=False)
  out_folder = idc_downloads_path
  !cat {selection_manifest} | gsutil -m cp -Ir $out_folder #download gcs_urls into output_folder
  #sort dicom donwload data per PatID/studyUID/serieUID
  # !python dicomsort/dicomsort.py -k -u {os.environ["IDC_Downloads"]} {os.environ["IDC_Downloads_Sorted"]}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
  # !mv {os.environ['IDC_Downloads_Sorted']+'/*'} {os.environ['prostatex_dicom']}#move sorted IDC DICOM data to prostatex_dicom folder
  !python dicomsort/dicomsort.py -k -u $out_folder {idc_downloads_sorted_path}/%PatientID/%StudyInstanceUID/%SeriesInstanceUID/%SOPInstanceUID.dcm
  in_mv = idc_downloads_sorted_path+'/*'
  out_mv = idc_dicom_path
  !mv $in_mv $out_mv
  #Convert to NIFTI
  convert_dcm_sorted(input_path=idc_dicom_path,
                  output_path=os.environ["prostatex_root"], idc_df=idc_t2_df)
  return idc_t2_df

In [47]:
def reformat_image_nnunet(patID, serieUID, images_nii_folder, out_path_folder):
  """
  formats input MR T2 data into nnunet ready format, caseID_Modality.nii.gz, where modality here is T2==0000
  """
  #reformats images to correct format,
  for mr_vol in glob.glob(os.path.join(images_nii_folder, f"{patID}_{serieUID}*.nii.gz")):#get MR T2 NIFTI volume
    serieUID = mr_vol.split('/')[-1].split("_")[1].replace(".nii.gz","")#.split(".")[0]
    patientID = mr_vol.split('/')[-1].split("_")[0]
    nnunet_idx = "0000" #if "T2" in mr_vol.split('/')[-2] else "0001"#0000 for T2 and 0001 for ADC
    nnunet_path = os.path.join(out_path_folder,
                                "_".join([patientID, serieUID, nnunet_idx]) + ".nii.gz")
    !cp $mr_vol $nnunet_path

In [48]:
def resample_preds(input_path_nnunet_preds="", input_path_t2_idc="", output_path="", suffix_ref=".nrrd"):
  """
  Resampling of nnunet_preds to reference MR T2 volumes used as input image
  input_path_nnunet_preds : str, folder containing nnunet preds
  input_path_t2_idc : str, folder containing NIFTI IDC MR T2 volumes
  output_path : str, folder output
  """
  for pred_path in sorted(glob.glob(os.path.join(input_path_nnunet_preds, "**", "*.nii.gz"), recursive=True)):
    search_t2_path = os.path.join(input_path_t2_idc, \
                                  f"{pred_path.split('/')[-1].split('_')[0]}_{pred_path.split('/')[-1].split('_')[1].replace('.nii.gz','')}*{suffix_ref}") #PatientID_SerieUID.nii.gz
                                      #get serieUID
    print(f"search path for {desired_grid} : {search_t2_path}")
    t2_path = glob.glob(search_t2_path, recursive=True)[0]
    print(f"{desired_grid}_path found : {t2_path}")
    print(f"pred path : {pred_path}")
    resample_args_to_t2_origin = {"input" : pred_path,#change to pred path if no largest_component_retrieval necessary
                          "output" : os.path.join(output_path,
                                                  f"{pred_path.split('/')[-1][:-7]}.nii.gz"),
                          "fixed" : t2_path,
                          "interpolation" : "nn"}

    pypla.resample(verbose = False, **resample_args_to_t2_origin)
    print()

In [49]:
def resample_seg(input_path_labelmap, input_path_nnunet_preds="", input_path_t2_idc="", output_path=""):
  """
  Resampling of nnunet_preds to reference MR T2 volumes used as input image
  input_path_nnunet_preds : str, folder containing nnunet preds
  input_path_t2_idc : str, folder containing NIFTI IDC MR T2 volumes
  output_path : str, folder output
  """
  pred_path = glob.glob(os.path.join(input_path_nnunet_preds, "*.nii.gz"))[0]
  for labelmap_file in sorted(glob.glob(os.path.join(input_path_labelmap, "*.nii.gz"))):
    search_t2_path = os.path.join(input_path_t2_idc, \
                                  f"{pred_path.split('/')[-1].split('_')[0]}_{pred_path.split('/')[-1].split('_')[1].replace('.nii.gz','')}*.nii.gz") #PatientID_SerieUID.nii.gz
                                      #get serieUID
    print(f"search path for {desired_grid} : {search_t2_path}")
    t2_path = glob.glob(search_t2_path, recursive=True)[0]
    print(f"{desired_grid}_path found : {t2_path}")
    print(f"pred path : {pred_path}")
    resample_args_to_t2_origin = {"input" : labelmap_file,#change to pred path if no largest_component_retrieval necessary
                          "output" : os.path.join(output_path,
                                                  f"{pred_path.split('/')[-1][:-7]}_resampled_seg.nii.gz"),
                          "fixed" : t2_path,
                          "interpolation" : "nn"}

    path_log = os.path.join(os.environ["logs"], 'log_pypla_res_pred' + pred_path.split('/')[-1].split('.')[0] + '.txt')
    !touch $path_log
    pypla.resample(verbose = False, **resample_args_to_t2_origin, path_to_log_file=path_log)
    print()

In [50]:
def resample_seg_idc(input_path_labelmap, input_path_nnunet_preds="", input_path_t2_idc="", output_path=""):
  pass
  #resample each label seg nii with original mr


In [51]:
def seg_nii_to_dicom(patID, studyUID, refSerieUID, json_metadata_dcm_path = "", input_path_nii="", input_path_dcm_idc="", output_path_root=""):
  """
  Convert of NIFTI binary segmentations of nnunet to SEG DICOM object
  idc_df : pandas.Dataframe, resulting from Bigquery
  input_path_nii : str, nnunet_preds folder
  input_path_dcm_idc : str,  folder containing DCM IDC MR T2 objects
  output_path_root : str, output_path root folder
  """
  assert os.path.exists(input_path_nii)
  assert os.path.exists(input_path_dcm_idc)
  !mkdir -p $output_path_root
  for nii_seg_pred in glob.glob(os.path.join(input_path_nii, '*.nii.gz')):
    patID = nii_seg_pred.split('/')[-1].split('_')[0]
    study_mr_t2_serieUID = nii_seg_pred.split('/')[-1].split('_')[1].replace(".nii.gz","")
    #find t2 dcm folder
    t2_dcm_folder = glob.glob(os.path.join(input_path_dcm_idc, patID, "**", study_mr_t2_serieUID))[0]
    #find seg dcm file
    # find nii seg folder == preds resampled
    assert os.path.exists(t2_dcm_folder)
    print('\nConverting...')
    print(f'pred nnunet processed : {nii_seg_pred}')
    print(f't2_dcm_folder : {t2_dcm_folder}')
    output_path = os.path.join(output_path_root, '_'.join([patID, study_mr_t2_serieUID])+'.dcm')
    #convert nii pred to dcm
    !itkimage2segimage --inputImageList $nii_seg_pred \
    --inputDICOMDirectory  $t2_dcm_folder \
    --inputMetadata  $json_metadata_dcm_path \
    --outputDICOM $output_path
    print("Done!")

## Evaluation functions

In [52]:
def compute_dice(pred_path, gt_path):
 #computation of dice score
  out_plast = !plastimatch dice --dice $gt_path $pred_path
  dice_score = [el[1] for el in out_plast.fields() if "DICE" in el[0]][0]#formatting by plastimatch dice output, retrieve of DSC
  return float(dice_score)

In [53]:
def compute_hsdff_95(pred_path, gt_path, labelID=1):
  out_plast = !plastimatch dice --hausdorff $gt_path $pred_path
  hsdff = [el[5] for el in out_plast.fields() if "Percent" in el[0] and "distance" in el[3]][0]#formatting by plastimatch hsdff output, retrieve of hsdff
  return float(hsdff)

In [54]:
def compute_hsdff_regular(pred_path, gt_path):
  #computation of dice score
  out_plast = !plastimatch dice --hausdorff $gt_path $pred_path
  hsdff = [el[3] for el in out_plast.fields() if "Hausdorff" in el[0] and "distance" in el[1]][0]#formatting by plastimatch hsdff output, retrieve of hsdff
  return float(hsdff)

In [55]:
#https://github.com/Jingnan-Jia/segmentation_metrics
def compute_avg_surface_dist(pred_path, gt_path):
  # for only one metrics
  image_gt = sitk.ReadImage(gt_path)
  image_pred = sitk.ReadImage(pred_path)
  gt_spacing = image_gt.GetSpacing()
  gt_size = image_gt.GetSize()
  new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(gt_size, gt_spacing, image_pred.GetSpacing())]
  gt_image_new  = sitk.Resample(image_gt, new_size, sitk.Transform(), sitk.sitkNearestNeighbor,
                        image_gt.GetOrigin(), image_pred.GetSpacing(), image_gt.GetDirection(), 0,
                        image_gt.GetPixelID())
  metrics = sg.write_metrics(labels=[1],  # exclude background if needed -- here only 1 label, prostate
                    gdth_img=gt_image_new,
                    pred_img=sitk.ReadImage(pred_path),
                    csv_file=None,
                    spacing=list(sitk.ReadImage(pred_path).GetSpacing()),
                    metrics='msd')
  return metrics

In [56]:
def compute_true_positive(pred_path, gt_path):
  out_plast = !plastimatch dice --all $gt_path $pred_path
  return [el[1] for el in out_plast.fields() if "TP" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [57]:
def compute_true_negative(pred_path, gt_path):
  out_plast = !plastimatch dice --all $gt_path $pred_path
  return [el[1] for el in out_plast.fields() if "TN" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [58]:
def compute_false_negative(pred_path, gt_path):
  out_plast = !plastimatch dice --all $gt_path $pred_path
  return [el[1] for el in out_plast.fields() if "FN" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [59]:
def compute_false_positive(pred_path, gt_path):
  out_plast = !plastimatch dice --all $gt_path $pred_path
  return [el[1] for el in out_plast.fields() if "FP" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [60]:
def compute_sensitivity(pred_path, gt_path):
  out_plast = !plastimatch dice --all $gt_path $pred_path
  return [el[1] for el in out_plast.fields() if "SE" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

In [61]:
def compute_specificity(pred_path, gt_path):
  out_plast = !plastimatch dice --all $gt_path $pred_path
  return [el[1] for el in out_plast.fields() if "SP" in el[0]][0]#formatting by plastimatch hsdff output, retrieve of hsdff

## nnUNet 3D shape features SR creation

In [62]:
def get_label_and_names_from_metadata_json(dicomseg_json):

  """Returns two lists containing the label values and the corresponding
     CodeMeaning values

  Inputs:
    dicomseg_json : metajson file

  Outputs:
    label_values  : label values from the metajson file
    label_names   : the corresponding CodeMeaning values

  """

  f = open(dicomseg_json)
  meta_json = json.load(f)

  print(meta_json)

  num_regions = len(meta_json['segmentAttributes'][0])
  print ('num_regions: ' + str(num_regions))

  label_values = []
  label_names = []
  for n in range(0,num_regions):
    # label_values.append(n)
    label_value = meta_json['segmentAttributes'][0][n]['labelID']
    label_name = meta_json['segmentAttributes'][0][n]['SegmentedPropertyTypeCodeSequence']['CodeMeaning']
    label_values.append(label_value)
    label_names.append(label_name)

  return label_values, label_names

In [63]:
def split_nii(input_file, output_directory, label_names):

  """Function to split a single multilabel nii into individual nii files. Used
     for pyradiomics feature extraction.

  Inputs:
    input_file       : input multi-label nii file
    output_directory : where to save the individual nii segments
    label_names      : the names of the labels that correspond to the order of
                       the values in the nii input_file

  Outputs:
    saves the individual nii files to the output_directory

  """

  if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

  # save with the values in the files
  nii = nib.load(input_file)
  header = nii.header
  img = nii.get_fdata()
  unique_labels = list(np.unique(img))
  unique_labels.remove(0) # remove the background

  # split and save
  num_labels = len(unique_labels)
  for n in range(0,num_labels):
    ind = np.where(img==unique_labels[n])
    vol = np.zeros((img.shape))
    vol[ind] = 1
    new_img = nib.Nifti1Image(vol, nii.affine, nii.header)
    output_filename = os.path.join(output_directory, label_names[n] + '.nii.gz')
    nib.save(new_img, output_filename)

  return

In [64]:
def compute_pyradiomics_3D_features(ct_nifti_path,
                                    label_values,
                                    label_names,
                                    split_pred_nifti_path,
                                    nnunet_shape_features_code_mapping_df):

  """Function to compute pyradiomics 3D features for each label in a nifti file.


  Inputs:
    ct_nifti_path            : the CT nifti file
    label_values             : the label value for each of the segments from the json file
    label_names              : the corresponding label name for each of the segments
    split_pred_nifti_path    : where to save the individual nii segments needed
                               for pyradiomics
    nnunet_shape_features_code_mapping_df : the df where we will obtain the
                                            list of the shape features to
                                            compute

  Outputs:
    Writes the features_csv_path_nnunet to disk.

  """

  # Get the names of the features from the nnunet_shape_features_code_mapping_df
  shape_features = list(nnunet_shape_features_code_mapping_df['shape_feature'].values)

  # Instantiate the extractor and modify the settings to keep the 3D shape features
  extractor = featureextractor.RadiomicsFeatureExtractor(correctMask=True, geometryTolerance=float(1e-2))
  extractor.settings['minimumROIDimensions'] = 3
  extractor.disableAllFeatures()
  extractor.enableFeaturesByName(shape=shape_features)

  # Calculate features for each label and create a dataframe
  num_labels = len(label_names)#len([f for f in os.listdir(split_pred_nifti_path) if f.endswith('.nii.gz')])
  df_list = []
  for n in range(0,num_labels):
    mask_path = os.path.join(split_pred_nifti_path, label_names[n] + '.nii.gz')
    # Run the extractor
    print(f"ct nifti path : {ct_nifti_path}")
    print(f" mask path : {mask_path}")
    result = extractor.execute(ct_nifti_path, mask_path) # dictionary
    # keep only the features we want
    # Get the corresponding label number -- all might not be present
    corresponding_label_value = label_values[label_names.index(label_names[n])]
    dict_keep = {'ReferencedSegment': corresponding_label_value,
                 'label_name': label_names[n]}
    keys_keep = [f for f in result.keys() if 'original_shape' in f]
    # Just keep the feature keys we want
    dict_keep_new_values = {key_keep: result[key_keep] for key_keep in keys_keep}
    dict_keep.update(dict_keep_new_values)
    df1 = pd.DataFrame([dict_keep])
    # change values of columns to remove original_shape_
    df1.columns = df1.columns.str.replace('original_shape_', '')
    # Append to the ReferencedSegment and label_name df
    df_list.append(df1)

  # concat all label features
  df = pd.concat(df_list)

  return df

In [65]:
def order_dicom_files_image_position(dcm_directory):
  """
  Orders the dicom files according to image position and orientation.

  Arguments:
    dcm_directory : input directory of dcm files to put in order

  Outputs:
    files_sorted   : dcm files in sorted order
    sop_all_sorted : the SOPInstanceUIDs in sorted order
    pos_all_sorted : the image position in sorted order

  """
  files = [os.path.join(dcm_directory,f) for f in os.listdir(dcm_directory)]

  num_files = len(files)

  pos_all = []
  sop_all = []

  for n in range(0,num_files):
    # read dcm file
    filename = files[n]
    ds = dcmread(filename)

    # get ImageOrientation (0020, 0037)
    # ImageOrientation = ds['0x0020','0x0037'].value
    ImageOrientation = ds.ImageOrientationPatient

    # get ImagePositionPatient (0020, 0032)
    # ImagePositionPatient = ds['0x0020','0x0032'].value
    ImagePositionPatient = ds.ImagePositionPatient

    # calculate z value
    x_vector = ImageOrientation[0:3]
    y_vector = ImageOrientation[3:]
    z_vector = np.cross(x_vector,y_vector)

    # multiple z_vector by ImagePositionPatient
    pos = np.dot(z_vector,ImagePositionPatient)
    pos_all.append(pos)

    # get the SOPInstanceUID
    # sop = ds['0x0008', '0x0018'].value
    sop = ds.SOPInstanceUID
    sop_all.append(sop)


  #----- order the SOPInstanceUID/files by z value ----#

  sorted_ind = np.argsort(pos_all)
  pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
  sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
  files_sorted = np.array(files)[sorted_ind.astype(int)]

  return files_sorted, sop_all_sorted, pos_all_sorted

In [66]:
def create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                         SOPInstanceUID_seg,
                                                         seg_file,
                                                         dcm_directory,
                                                         segments_code_mapping_df,
                                                         shape_features_code_mapping_df,
                                                         df_features,
                                                         SegmentAlgorithmName
                                                         ):

  """Function that creates the metajson necessary for the creation of a
  structured report from a pandas dataframe of label names and features for
  each.

  Inputs:
    SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                      file
    SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
    seg_file                        : filename of SEG DCM file
    dcm_directory                   : ct directory that will be sorted in
                                      terms of axial ordering according to the
                                      ImagePositionPatient and ImageOrientation
                                      fields
    segments_code_mapping_df        : dataframe that holds the names of the
                                      segments and the associated code values etc.
    shape_features_code_mapping_df  : dataframe that holds the names of the
                                      features and the associated code values etc.
    df_features                     : a pandas dataframe holding the segments and a
                                      set of 3D shape features for each
    SegmentAlgorithmName            : the name of the algorithm used to create the
                                      segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report
  """

  # --- Get the version number for the pyradiomics package --- #

  pyradiomics_version_number = str(radiomics.__version__)

  # --- Sort the dcm files first according to --- #
  # --- ImagePositionPatient and ImageOrientation --- #

  files_sorted, sop_all_sorted, pos_all_sorted = order_dicom_files_image_position(dcm_directory)
  files_sorted = [os.path.basename(f) for f in files_sorted]

  # --- Create the header for the json --- #

  inputMetadata = {}
  inputMetadata["@schema"]= "https://raw.githubusercontent.com/qiicr/dcmqi/master/doc/schemas/sr-tid1500-schema.json#"
  # inputMetadata["SeriesDescription"] = "Measurements"
  inputMetadata["SeriesDescription"] = SegmentAlgorithmName + '_' + "Measurements"
  inputMetadata["SeriesNumber"] = "1001"
  inputMetadata["InstanceNumber"] = "1"

  inputMetadata["compositeContext"] = [seg_file] # not full path

  inputMetadata["imageLibrary"] = files_sorted # not full path

  # inputMetadata["observerContext"] = {
  #                                     "ObserverType": "PERSON",
  #                                     "PersonObserverName": "Reader1"
  #                                   }
  # inputMetadata["observerContext"] = {
  #                     "ObserverType": "DEVICE",
  #                     "DeviceObserverName": "pyradiomics",
  #                     "DeviceObserverModelName": "v3.0.1"
  #                   }
  inputMetadata["observerContext"] = {
                      "ObserverType": "DEVICE",
                      "DeviceObserverName": "pyradiomics",
                      "DeviceObserverModelName": pyradiomics_version_number
                    }

  inputMetadata["VerificationFlag"]  = "UNVERIFIED"
  inputMetadata["CompletionFlag"] =  "COMPLETE"
  inputMetadata["activitySession"] = "1"
  inputMetadata["timePoint"] = "1"

  # ------------------------------------------------------------------------- #
  # --- Create the measurement_dict for each segment - holds all features --- #

  measurement = []

  # --- Now create the dict for all features and all segments --- #

  # --- Loop over the number of segments --- #

  # number of rows in the df_features
  num_segments = df_features.shape[0]

  # Array of dictionaries - one dictionary for each segment
  measurement_across_segments_combined = []

  for segment_id in range(0,num_segments):

    ReferencedSegment = df_features['ReferencedSegment'].values[segment_id]
    FindingSite = df_features['label_name'].values[segment_id]

    print('segment_id: ' + str(segment_id))
    print('ReferencedSegment: ' + str(ReferencedSegment))
    print('FindingSite: ' + str(FindingSite))

    # --- Create the dict for the Measurements group --- #
    TrackingIdentifier = "Measurements group " + str(ReferencedSegment)

    segment_row = segments_code_mapping_df[segments_code_mapping_df["segment"] == FindingSite]
    # print(segment_row)

    my_dict = {
      "TrackingIdentifier": str(TrackingIdentifier),
      "ReferencedSegment": int(ReferencedSegment),
      "SourceSeriesForImageSegmentation": str(SeriesInstanceUID),
      "segmentationSOPInstanceUID": str(SOPInstanceUID_seg),
      "Finding": {
        "CodeValue": "113343008",
        "CodingSchemeDesignator": "SCT",
        "CodeMeaning": "Organ"
      },
      "FindingSite": {
        "CodeValue": str(segment_row["FindingSite_CodeValue"].values[0]),
        "CodingSchemeDesignator": str(segment_row["FindingSite_CodingSchemeDesignator"].values[0]),
        "CodeMeaning": str(segment_row["FindingSite_CodeMeaning"].values[0])
      }
    }

    measurement = []
    # number of features - number of columns in df_features - 2 (label_name and ReferencedSegment)
    num_values = len(df_features.columns)-2

    feature_list = df_features.columns[2:] # remove first two


    # For each measurement per region segment
    for n in range(0,num_values):
      measurement_dict = {}
      row = df_features.loc[df_features['label_name'] == FindingSite]
      feature_row = shape_features_code_mapping_df.loc[shape_features_code_mapping_df["shape_feature"] == feature_list[n]]
      value = str(np.round(row[feature_list[n]].values[0],3))
      measurement_dict["value"] = value
      measurement_dict["quantity"] = {}
      measurement_dict["quantity"]["CodeValue"] = str(feature_row["quantity_CodeValue"].values[0])
      measurement_dict["quantity"]["CodingSchemeDesignator"] = str(feature_row["quantity_CodingSchemeDesignator"].values[0])
      measurement_dict["quantity"]["CodeMeaning"] = str(feature_row["quantity_CodeMeaning"].values[0])
      measurement_dict["units"] = {}
      measurement_dict["units"]["CodeValue"] = str(feature_row["units_CodeValue"].values[0])
      measurement_dict["units"]["CodingSchemeDesignator"] = str(feature_row["units_CodingSchemeDesignator"].values[0])
      measurement_dict["units"]["CodeMeaning"] = str(feature_row["units_CodeMeaning"].values[0])
      measurement_dict["measurementAlgorithmIdentification"] = {}
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmName"] = "pyradiomics"
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmVersion"] = str(pyradiomics_version_number)
      measurement.append(measurement_dict)

    measurement_combined_dict = {}
    measurement_combined_dict['measurementItems'] = measurement # measurement is an array of dictionaries

    output_dict_one_segment = {**my_dict, **measurement_combined_dict}

    # append to array for all segments

    measurement_across_segments_combined.append(output_dict_one_segment)

  # --- Add the measurement data --- #

  inputMetadata["Measurements"] = {}
  inputMetadata["Measurements"] = measurement_across_segments_combined

  return inputMetadata

In [67]:
def save_structured_report_for_shape_features(SeriesInstanceUID,
                                              SOPInstanceUID_seg,
                                              pred_dicomseg_path,
                                              dicomseg_json_path,
                                              dcm_directory,
                                              pred_nifti_path,
                                              nnunet_base_path,
                                              ct_nifti_path,
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path,
                                              SegmentAlgorithmName
                                              ):

  """ This function creates the SR necessary for the nnUNet shape features

  Inputs:
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                    file
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
  pred_dicomseg_path              : filename of DICOM SEG file
  dicomseg_json_path              : json file for DICOM SEG file
  dcm_directory                   : list of ct files that will be sorted in
                                    terms of axial ordering according to the
                                    ImagePositionPatient and ImageOrientation
                                    fields
  pred_nifti_path                 : predictions in nifti format
  nnunet_base_path                : path to hold the split nifti files
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the
                                    features and the associated code values etc.
  sr_json_path                    : the path that the metajson for the SR for
                                    the 3D shape features will be saved
  sr_path                         : the path that the SR for the 3D shape
                                    features will be saved
  SegmentAlgorithmName            : the name of the algorithm used to create the
                                    segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = get_label_and_names_from_metadata_json(dicomseg_json_path)

  # # --- split the multilabel nifti into individual files --- #
  split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  if not os.path.isdir(split_pred_nii_path):
    os.mkdir(split_pred_nii_path)
  split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # --- compute features and save csv for each region --- #
  # if not os.path.isdir(os.environ["nnUNet_preds_features"]):
  #   os.mkdir(os.environ["nnUNet_preds_features"])
  df_features = compute_pyradiomics_3D_features(ct_nifti_path,
                                                label_values,
                                                label_names,
                                                split_pred_nii_path,
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')

  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  # !rm $split_pred_nii_path/*
  # remove csv
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path,
                                                                       dcm_directory,
                                                                       segments_code_mapping_df,
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features,
                                                                       SegmentAlgorithmName)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- #
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json))
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputCompositeContextDirectory $inputCompositeContextDirectory \
                --inputMetadata $inputMetadata_json
  print ('wrote out SR for shape features')

  return

In [68]:
def save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID,
                                              label_values,
                                              label_names,
                                              SOPInstanceUID_seg,
                                              pred_dicomseg_path,
                                              dicomseg_json_path,
                                              dcm_directory,
                                              pred_nifti_path,
                                              split_pred_nii_path,
                                              ct_nifti_path,
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path,
                                              SegmentAlgorithmName
                                              ):

  """ This function creates the SR necessary for the nnUNet shape features

  Inputs:
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                    file
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
  pred_dicomseg_path              : filename of DICOM SEG file
  dicomseg_json_path              : json file for DICOM SEG file
  dcm_directory                   : list of ct files that will be sorted in
                                    terms of axial ordering according to the
                                    ImagePositionPatient and ImageOrientation
                                    fields
  pred_nifti_path                 : predictions in nifti format
  nnunet_base_path                : path to hold the split nifti files
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the
                                    features and the associated code values etc.
  sr_json_path                    : the path that the metajson for the SR for
                                    the 3D shape features will be saved
  sr_path                         : the path that the SR for the 3D shape
                                    features will be saved
  SegmentAlgorithmName            : the name of the algorithm used to create the
                                    segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = label_values, label_names#get_label_and_names_from_metadata_json(dicomseg_json_path)

  # # --- split the multilabel nifti into individual files --- #
  # split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  # if not os.path.isdir(split_pred_nii_path):
  #   os.mkdir(split_pred_nii_path)
  # split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # # --- compute features and save csv for each region --- #
  # if not os.path.isdir(os.environ["nnUNet_preds_features"]):
  #   os.mkdir(os.environ["nnUNet_preds_features"])
  df_features = compute_pyradiomics_3D_features(ct_nifti_path,
                                                label_values,
                                                label_names,
                                                split_pred_nii_path,
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')

  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  !rm $split_pred_nii_path/*
  # remove csv
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path,
                                                                       dcm_directory,
                                                                       segments_code_mapping_df,
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features,
                                                                       SegmentAlgorithmName)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- #
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json))
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputCompositeContextDirectory $inputCompositeContextDirectory \
                --inputMetadata $inputMetadata_json
  print ('wrote out SR for shape features')

  return

In [69]:
def save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID,
                                              label_values,
                                              label_names,
                                              SOPInstanceUID_seg,
                                              pred_dicomseg_path,
                                              dicomseg_json_path,
                                              dcm_directory,
                                              pred_nifti_path,
                                              split_pred_nii_path,
                                              ct_nifti_path,
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path,
                                              SegmentAlgorithmName
                                              ):

  """ This function creates the SR necessary for the nnUNet shape features

  Inputs:
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT
                                    file
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file
  pred_dicomseg_path              : filename of DICOM SEG file
  dicomseg_json_path              : json file for DICOM SEG file
  dcm_directory                   : list of ct files that will be sorted in
                                    terms of axial ordering according to the
                                    ImagePositionPatient and ImageOrientation
                                    fields
  pred_nifti_path                 : predictions in nifti format
  nnunet_base_path                : path to hold the split nifti files
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the
                                    features and the associated code values etc.
  sr_json_path                    : the path that the metajson for the SR for
                                    the 3D shape features will be saved
  sr_path                         : the path that the SR for the 3D shape
                                    features will be saved
  SegmentAlgorithmName            : the name of the algorithm used to create the
                                    segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = label_values, label_names#get_label_and_names_from_metadata_json(dicomseg_json_path)

  # # --- split the multilabel nifti into individual files --- #
  # split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  # if not os.path.isdir(split_pred_nii_path):
  #   os.mkdir(split_pred_nii_path)
  # split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # # --- compute features and save csv for each region --- #
  # if not os.path.isdir(os.environ["nnUNet_preds_features"]):
  #   os.mkdir(os.environ["nnUNet_preds_features"])
  df_features = compute_pyradiomics_3D_features(ct_nifti_path,
                                                label_values,
                                                label_names,
                                                split_pred_nii_path,
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')

  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  !rm $split_pred_nii_path/*
  # remove csv
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID,
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path,
                                                                       dcm_directory,
                                                                       segments_code_mapping_df,
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features,
                                                                       SegmentAlgorithmName)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- #
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json))
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputMetadata $inputMetadata_json
                # --inputCompositeContextDirectory $inputCompositeContextDirectory \ ##fails for now
  print ('wrote out SR for shape features')

  return

# Data selection, downloading and structuring

We want to select here the imaging metadata and expert annotations metadata from IDC

In order to use data hosted by IDC effectively, you will need to utilize metadata to navigate what data is available and to select specific files that are relevant in your analysis. The main metadata table you will need for this purpose is the [`bigquery-public-data.idc_current.dicom_all`](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=idc_current&t=dicom_all&page=table) table.

This query has one row per file hosted by IDC. All of IDC data is in DICOM format, and each of the rows in this table will have all of the DICOM attributes extracted from a given file. It will also have various columns containing non-DICOM metadata, such as the name of the collection where the file is included, size of the file, and URL that can be used to retrieve that file.

To query IDC BigQuery tables, you can use one of the following approaches:
1. `%%bigquery` magic will allow you to define your query in plain SQL, and load the result of the query into a Pandas dataframe.
2. [BigQuery Python API](https://googleapis.dev/python/bigquery/latest/index.html) is more flexible in allowing you to parameterize your query.
3. [Google Cloud BigQuery console](https://console.cloud.google.com/bigquery) is very convenient for interactive query exploration of tables.
4. [`gcloud bq`](https://cloud.google.com/bigquery/docs/bq-command-line-tool) is the command line tool that comes as part of [Cloud SDK](https://cloud.google.com/sdk) and is convenient for scripting interactions from the shell. Cloud SDK is preinstalled on Colab.

In the following cells we will utilize `%%bigquery`, Python BigQuery SDK and BigQuery console for working with IDC BigQuery tables.

First, to verify that you are authenticated, and your project ID is working, let's run a test query against IDC BigQuery table to get the summary statistics about the  of data available in IDC.


Given `SeriesInstanceUID` value identifying the image series, we can query the IDC metadata table to get the list of files (defined by the Google Storage URLs) corresponding to this series.

All of the DICOM metadata for each of the DICOM files is available in the BigQuery table we will be querying. We will get not just the `gcs_url`, but also identifiers for the Study, Series and Instance, to better understand organization of data, and since `StudyInstanceUID` will be handy later when we get to the visualization of the data.

In [70]:
# selection_query = f"""
#   SELECT
#     dc_all.gcs_url,
#     dc_all.SOPInstanceUID,
#     dc_all.SeriesInstanceUID,
#     dc_all.StudyInstanceUID,
#     dc_all.PatientID,
#     dc_all.collection_id,
#     dc_all.Modality
#   FROM
#     `bigquery-public-data.idc_v15.dicom_all` AS dc_all
#   WHERE
#     dc_all.collection_id = 'prostate_mri_us_biopsy'
#     AND dc_all.Modality = 'MR'
#   ORDER BY
#     PatientID"""
# selection_result = bq_client.query(selection_query)
# selection_df = selection_result.result().to_dataframe()

In [71]:
!wget -N https://raw.githubusercontent.com/ccosmin97/idc-prostate-mri-analysis/main/data/idc_manifests/idc_images_prostate_mri_us_biopsy_prostate.csv
selection_df = pd.read_csv("idc_images_prostate_mri_us_biopsy_prostate.csv")

--2024-06-13 03:49:30--  https://raw.githubusercontent.com/ccosmin97/idc-prostate-mri-analysis/main/data/idc_manifests/idc_images_prostate_mri_us_biopsy_prostate.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20659636 (20M) [text/plain]
Saving to: ‘idc_images_prostate_mri_us_biopsy_prostate.csv’

idc_images_prostate 100%[===================>]  19.70M  --.-KB/s    in 0.1s    

Last-modified header missing -- time-stamps turned off.
2024-06-13 03:49:31 (158 MB/s) - ‘idc_images_prostate_mri_us_biopsy_prostate.csv’ saved [20659636/20659636]



In [72]:
# selection_query = f"""
#   SELECT
#     gcs_url,
#     SOPInstanceUID,
#     SeriesInstanceUID,
#     StudyInstanceUID,
#     PatientID,
#     collection_id,
#     Modality,
#     ReferencedSeriesSequence[SAFE_OFFSET(0)].SeriesInstanceUID as RefSerieUID
#   FROM
#     `bigquery-public-data.idc_v18.dicom_all`
#   WHERE
#     collection_id = 'prostate_mri_us_biopsy'
#     AND SegmentSequence[SAFE_OFFSET(0)].SegmentedPropertyTypeCodeSequence[SAFE_OFFSET(0)].CodeMeaning = 'Prostate'
#     AND Modality='SEG'
#     AND SeriesDescription = 'Segmentation of prostate'
#     """
# selection_result = bq_client.query(selection_query)
# selection_t2_seg_prostate_df = selection_result.result().to_dataframe()

In [73]:
# selection_t2_seg_prostate_df.to_csv("idc_expert_seg_prostate_mri_us_biopsy_prostate.csv", index=None)
!wget -N https://raw.githubusercontent.com/ccosmin97/idc-prostate-mri-analysis/main/data/idc_manifests/idc_expert_seg_prostate_mri_us_biopsy_prostate.csv
selection_t2_seg_prostate_df = pd.read_csv("idc_expert_seg_prostate_mri_us_biopsy_prostate.csv")

--2024-06-13 03:49:31--  https://raw.githubusercontent.com/ccosmin97/idc-prostate-mri-analysis/main/data/idc_manifests/idc_expert_seg_prostate_mri_us_biopsy_prostate.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391805 (383K) [text/plain]
Saving to: ‘idc_expert_seg_prostate_mri_us_biopsy_prostate.csv’

idc_expert_seg_pros 100%[===================>] 382.62K  --.-KB/s    in 0.03s   

Last-modified header missing -- time-stamps turned off.
2024-06-13 03:49:31 (12.0 MB/s) - ‘idc_expert_seg_prostate_mri_us_biopsy_prostate.csv’ saved [391805/391805]



In [74]:
len(selection_t2_seg_prostate_df.RefSerieUID.unique())

1017

In [75]:
len(selection_df.SeriesInstanceUID.unique())

1017

In [76]:
len(selection_df.SeriesInstanceUID.unique())

1017

# Download json metadata template for conversion of AI model into DICOM

In [77]:
!wget -N https://raw.githubusercontent.com/ccosmin97/idc-prostate-mri-analysis/main/data/dicom_utils/metadata_jsons_dcmqi/pred_metadata_prostate158_all.json
seg_dcm_metadata_json_file = "pred_metadata_prostate158_all.json"

--2024-06-13 03:49:31--  https://raw.githubusercontent.com/ccosmin97/idc-prostate-mri-analysis/main/data/dicom_utils/metadata_jsons_dcmqi/pred_metadata_prostate158_all.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2445 (2.4K) [text/plain]
Saving to: ‘pred_metadata_prostate158_all.json’

pred_metadata_prost 100%[===================>]   2.39K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-06-13 03:49:32 (62.3 MB/s) - ‘pred_metadata_prostate158_all.json’ saved [2445/2445]



# Download and install MONAI and pre-trained model

In [78]:
!wget https://zenodo.org/record/7040585/files/anatomy.pt?download=1 -O {os.environ['monai_workspace_model_path']}
# !zenodo_get 7040585
# !mv ./anatomy.pt {os.environ["monai_workspace_model_dir"]}/anatomy.pt

--2024-06-13 03:49:32--  https://zenodo.org/record/7040585/files/anatomy.pt?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/7040585/files/anatomy.pt [following]
--2024-06-13 03:49:32--  https://zenodo.org/records/7040585/files/anatomy.pt
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 152810901 (146M) [application/octet-stream]
Saving to: ‘monai_workspace/models/anatomy.pt’

monai_workspace/mod 100%[===================>] 145.73M  22.1MB/s    in 7.9s    

2024-06-13 03:49:41 (18.5 MB/s) - ‘monai_workspace/models/anatomy.pt’ saved [152810901/152810901]



We process each SerieUID one at a time, we first retrieve serieUIDs processed in GCP bucket and subtract that amount from the serieUIDs retrieved from IDC MR T2 series, obtaining the non processed serieUID_current_lst.

If that is not the desired behaviour, change serieUID_current_lst below to list((selection_df.SeriesInstanceUID.unique())

# Main loop

In [79]:
def reset_folders():
  for key, path in os.environ.items():
    check_patterns = [True for el in ["prostatex", "IDC_Downloads", "IDC_Downloads_Sorted", "IDC_IMG_Downloads", "IDC_IMG_Downloads_Sorted" \
                                      "prostatex_dicom", "prostatex_root",
                                       "stl_labelmap", "mri_us_biopsy_analysis", "IDC", "monai_workspace_eval", "monai_workspace_dataset"] if el in key]
    if True in check_patterns:
      !rm -rf $path
      !mkdir -p $path

In [80]:
selection_df.SeriesInstanceUID.unique()

array(['1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221',
       '1.3.6.1.4.1.14519.5.2.1.186749128823666050588710146976747922803',
       '1.3.6.1.4.1.14519.5.2.1.134581986918909360753889591939462644248',
       ...,
       '1.3.6.1.4.1.14519.5.2.1.330314825479824993086684610534464934728',
       '1.3.6.1.4.1.14519.5.2.1.270992234048173170117855949480996332900',
       '1.3.6.1.4.1.14519.5.2.1.160737845666915287886280362832734640272'],
      dtype=object)

In [81]:
#add to BQ table
#whole process
for serieUID_current in  selection_df.SeriesInstanceUID.unique()[0:1]:
  #reset processing folders
  reset_folders()
  #download idc data
  idc_df = download_idc_data_serie_uid(idc_t2_df=selection_df[selection_df.SeriesInstanceUID \
                                                          == serieUID_current], idc_downloads_path=os.environ["IDC_IMG_Downloads"],
                                      idc_downloads_sorted_path=os.environ["IDC_IMG_Downloads_Sorted"],
                                      idc_dicom_path=os.environ["prostatex_dicom"],
                                      idc_nii_path=os.environ["prostatex_nii"])
  ##reformat idc input data
  patID = idc_df[idc_df["SeriesInstanceUID"] == serieUID_current].PatientID.unique()[0]
  reformat_image_nnunet(patID, serieUID_current, images_nii_folder = os.environ["prostatex_nii"], out_path_folder=os.environ["prostatex_nii_formatted"])
  #setup monai workspace
  test_df = pd.DataFrame(columns=["t2", "t2_label"])#t2 is serieUID of reference t2 image
  !cp -r {os.environ['prostatex_nii_formatted']}/*.nii.gz {os.environ["monai_workspace_dataset"]}/
  test_df.to_csv(os.path.join(os.environ["monai_workspace_dataset"], "test.csv"))
  inference_json = json.load(open("/content/model-zoo/models/prostate_mri_anatomy/configs/inference.json"))#path to downloaded bundle from git monai bundles prostate158
  #setup variables
  inference_json["bundle_root"] = os.environ["monai_workspace"]
  inference_json["output_dir"] = os.environ["monai_workspace_eval"]
  inference_json["dataset_dir"] = os.environ["monai_workspace_dataset"]
  inference_json["handlers"][0]["load_path"] = os.environ["monai_workspace_model_path"]
  datalist_json = glob.glob(os.path.join(os.environ["monai_workspace_dataset"], "*.nii.gz"))
  inference_json["datalist"] = datalist_json
  with open(f"{os.environ['monai_workspace']}/inference.json", 'w') as fp:
      json.dump(inference_json, fp)
  #inference
  config = ConfigParser()
  config.read_config(f"{os.environ['monai_workspace']}/inference.json")
  now = time.time()
  !python -m monai.bundle run evaluating --meta_file /content/model-zoo/models/prostate_mri_anatomy/configs/metadata.json --config_file {os.environ['monai_workspace']}/inference.json
  time_pred = int(time.time() - now)
  # # #resample preds data
  resample_preds(input_path_nnunet_preds=os.environ["monai_workspace_eval"],\
                  input_path_t2_idc=os.environ["prostatex_nrrd"], output_path=os.environ["monai_workspace_eval_resampled"])
  #split preds data for analysis -- in case of multiple segments
  split_seg_nii(input_labelmap=glob.glob(f"{os.environ['monai_workspace_eval_resampled']}/*.nii.gz")[0],
                output_folder_path= os.environ["monai_workspace_eval_resampled_split"],dic_labelID={"Transition zone of the prostate" : 1, "Peripheral zone of the prostate" : 2})
  #combine pz and central gland to form prostate
  combine_prostate_regions(input_path_nii_pred=glob.glob(f"{os.environ['monai_workspace_eval_resampled']}/*.nii.gz")[0],
  output_path_folder=os.environ["monai_workspace_eval_resampled_split"], filename="Prostate.nii.gz")
  # #convert nnunet_preds to dicom
  patID = selection_df[selection_df["SeriesInstanceUID"] == serieUID_current].PatientID.unique()[0]
  studyUID = selection_df[selection_df["SeriesInstanceUID"] == serieUID_current].StudyInstanceUID.unique()[0]
  #Conversion to dcm
  ## reformat labelID to 1,23 in monai_split_dir
  change_labelID_seg_nii(glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/Transition*.nii.gz")[0],
                         1, os.environ['monai_workspace_eval_resampled_split_labelID'])
  change_labelID_seg_nii(glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/Peripheral*.nii.gz")[0],
                         2, os.environ['monai_workspace_eval_resampled_split_labelID'])
  change_labelID_seg_nii(glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/Prostate.nii.gz")[0],
                         3, os.environ['monai_workspace_eval_resampled_split_labelID'])
  list_seg_labelID = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/*.nii.gz")
  t2_dcm_folder = glob.glob(f"{os.environ['prostatex_dicom']}/{patID}/{studyUID}/{serieUID_current}")[0]
  output_path = f"{os.environ['monai_workspace_eval_resampled_dcm']}/{patID}_{serieUID_current}.dcm"
  prostate_nii = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/Prostate.nii.gz")[0]
  tz_nii = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/Transition*.nii.gz")[0]
  pz_nii = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split_labelID']}/Peripheral*.nii.gz")[0]
  !itkimage2segimage --inputImageList '{tz_nii}','{pz_nii}','{prostate_nii}' \
    --inputDICOMDirectory  $t2_dcm_folder \
    --inputMetadata  $seg_dcm_metadata_json_file \
    --outputDICOM $output_path
  #convert dicom nnunet preds back to nifti
  convert_seg_to_nii(input_path=glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/*.dcm")[0], \
                    output_path=os.environ['monai_workspace_eval_resampled_dcm_to_nifti'])
  #download SEG IDC DATA
  idc_seg_data_df = selection_t2_seg_prostate_df[selection_t2_seg_prostate_df.RefSerieUID \
                                                           == serieUID_current]
  download_idc_data_serie_uid_seg(df_in=idc_seg_data_df,
                                       out_path=os.environ["prostatex_dicom_seg"],
                                      idc_downloads_path=os.environ["IDC_Downloads"],
                                  idc_downloads_sorted_path=os.environ["IDC_Downloads_Sorted"])
  convert_seg_to_nii(input_path=glob.glob(f"{os.environ['prostatex_dicom_seg']}/**/*.dcm", recursive=True)[0], \
                    output_path=os.environ['prostatex_nii_seg'])
  # # reformat seg to codeMeaning.nii.gz
  t2_nii_path = glob.glob(os.path.join(os.environ["prostatex_nii"], f"{patID}_{serieUID_current}*.nii.gz"))[0]
  resample_idc_seg_nii(os.environ['prostatex_nii_seg'], os.environ["prostatex_nii_seg_resampled"], t2_nii_path)
  change_labelID_seg_nii(glob.glob(f"{os.environ['prostatex_nii_seg_resampled']}/*.nii.gz")[0],
                         1, os.environ['prostatex_nii_seg_resampled_formatted'])
  seg_gold_serieUID = str(pydicom.dcmread(glob.glob(f"{os.environ['prostatex_dicom_seg']}/**/*.dcm", recursive=True)[0]).SeriesInstanceUID)
  ai_serieUID = str(pydicom.dcmread(glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/**/*.dcm", recursive=True)[0]).SeriesInstanceUID)
  seg_pred_nnunet_sopInstanceUID = str(pydicom.dcmread(glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/*.dcm")[0]).SOPInstanceUID)
  dicomseg_json_path_modified = glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm_to_nifti']}/*.json")[0]
  for expert_nii in glob.glob(f"{os.environ['prostatex_nii_seg_resampled']}/*.nii.gz"):
    assert sorted(np.unique(sitk.GetArrayFromImage(sitk.ReadImage(expert_nii))))[1] == 1
    ai_nii_path = glob.glob(f"{os.environ['monai_workspace_eval_resampled_split']}/Prostate.nii.gz")[0]
    assert sorted(np.unique(sitk.GetArrayFromImage(sitk.ReadImage(ai_nii_path))))[1] == 1
    rows_to_insert = [
      {"refSerieUID": serieUID_current,
      "ai_serieUID" : ai_serieUID,
      "gold_serieUID" : seg_gold_serieUID,
      "segment" : "Prostate",
      "dsc": compute_dice(ai_nii_path, expert_nii),
      "hsdff" : compute_hsdff_regular(ai_nii_path, expert_nii),
      "hsdff_95" : compute_hsdff_95(ai_nii_path, expert_nii),
      "tp" : compute_true_positive(ai_nii_path, expert_nii),
      "fp" : compute_false_positive(ai_nii_path, expert_nii),
      "tn" : compute_true_negative(ai_nii_path, expert_nii),
      "fn" : compute_false_negative(ai_nii_path, expert_nii),
      "se" : compute_sensitivity(ai_nii_path, expert_nii),
      "sp" : compute_specificity(ai_nii_path, expert_nii),
      "prediction_time" : float(time_pred)
      }
      ]
    try:
      asd  = round(float(compute_avg_surface_dist(ai_nii_path, expert_nii)[0]['msd'][0]),2)
    except:
      asd = -999.
    rows_to_insert[0]["asd"] = asd
    print(rows_to_insert)
  # #SR DICOM
  # # #get dcm dir
  ct_nifti_path = glob.glob(f"{os.environ['prostatex_nrrd']}/{patID}_{serieUID_current}*.nrrd")[0]
  pred_path = glob.glob(f"{os.environ['monai_workspace_eval_resampled']}/*.nii.gz")[0]
  nnunet_features_metajson = save_structured_report_for_shape_features_idc_seg(SeriesInstanceUID=serieUID_current,
      SOPInstanceUID_seg=seg_pred_nnunet_sopInstanceUID,
      label_values=[1,2,3],
      label_names = ["Transition zone of the prostate", "Peripheral zone of the prostate","Prostate"],
      pred_dicomseg_path=glob.glob(f"{os.environ['monai_workspace_eval_resampled_dcm']}/*.dcm")[0],
      dicomseg_json_path=dicomseg_json_path_modified,
      dcm_directory=t2_dcm_folder,
      pred_nifti_path=pred_path,
      split_pred_nii_path=os.environ["monai_workspace_eval_resampled_split"],
      ct_nifti_path=ct_nifti_path,
      segments_code_mapping_df=idc_seg_segments_code_mapping_df,
      shape_features_code_mapping_df=nnunet_shape_features_code_mapping_df,
      sr_json_path=os.path.join(os.environ['monai_workspace_eval_sr_json'], serieUID_current + '_SR.json'),
      sr_path=os.path.join(os.environ['monai_workspace_eval_sr'], serieUID_current + '_SR.dcm'),
      SegmentAlgorithmName='Prostate158-measurements')
  # # ###### UPLOAD RESULTS
  out_nnunet_dcm_seg = os.environ['monai_workspace_eval_resampled_dcm']
  out_nnunet_sr_seg = os.environ['monai_workspace_eval_sr']

Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/01423c3f-9b1d-4ed5-8e29-604243936ab0.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/2cb79dfd-cb00-4860-bdd1-9819aab4145a.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/a9541bd2-e168-4dbb-9edf-65ddc33a750f.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/e5efad5b-51cc-4d4c-a2a7-d0a75705e4ac.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/e2bf0f1b-fecb-47b6-b371-888fdf6ecdde.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/e7f2d7dc-1c01-4a8e-a48f-f8aeba67621b.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/e15e1988-de14-4bfa-ac6b-76fe35529e94.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/e887b0eb-8cca-44f6-9cf5-bbf793820204.dcm...
Copying gs://public-datasets-idc/984f2b7e-54d8-42b7-afc0-af51eb8550e3/00f1c3aa-1be8-445c

INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}
INFO:radiomics.featureextractor:Applying custom setting overrides: {'correctMask': True, 'geometryTolerance': 0.01}
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


[{'refSerieUID': '1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221', 'ai_serieUID': '1.2.276.0.7230010.3.1.3.481037312.6796.1718250627.514348', 'gold_serieUID': '1.2.276.0.7230010.3.1.3.0.40360.1698534875.387515', 'segment': 'Prostate', 'dsc': 0.910396, 'hsdff': 4.898294, 'hsdff_95': 0.75, 'tp': '64502', 'fp': '2946', 'tn': '3854961', 'fn': '9751', 'se': '0.868679', 'sp': '0.999236', 'prediction_time': 28.0, 'asd': 6.42}]
ct nifti path : /content/prostateX/nrrd/Prostate-MRI-US-Biopsy-0001_1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221.nrrd
 mask path : monai_workspace/eval_resampled_split/Transition zone of the prostate.nii.gz


INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


ct nifti path : /content/prostateX/nrrd/Prostate-MRI-US-Biopsy-0001_1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221.nrrd
 mask path : monai_workspace/eval_resampled_split/Peripheral zone of the prostate.nii.gz


INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


ct nifti path : /content/prostateX/nrrd/Prostate-MRI-US-Biopsy-0001_1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221.nrrd
 mask path : monai_workspace/eval_resampled_split/Prostate.nii.gz


INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image


created df_features
segment_id: 0
ReferencedSegment: 1
FindingSite: Transition zone of the prostate
segment_id: 1
ReferencedSegment: 2
FindingSite: Peripheral zone of the prostate
segment_id: 2
ReferencedSegment: 3
FindingSite: Prostate
created SR json for shape features
wrote out json for shape features
inputImageLibraryDirectory: /content/prostateX/dicom/Prostate-MRI-US-Biopsy-0001/1.3.6.1.4.1.14519.5.2.1.85548304921965658367726869399297351743/1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221
outputDICOM: monai_workspace/structured_reports/1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221_SR.dcm
inputCompositeContextDirectory: Prostate-MRI-US-Biopsy-0001_1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221.dcm
inputMetadata_json: monai_workspace/structured_reports_json/1.3.6.1.4.1.14519.5.2.1.266717969984343981963002258381778490221_SR.json
dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
Total measurement groups